In [97]:
import pip
pip.main(["install","plotly"])
#!pip3 install plotly

  Running setup.py bdist_wheel for plotly: started
  Running setup.py bdist_wheel for plotly: finished with status 'done'
  Stored in directory: C:\Users\rinlin\AppData\Local\pip\Cache\wheels\05\fc\df\cd14e7f2faf8b90c9dbdeb265f0ceb89cbc80e218eae5ffc3e
Successfully built plotly
  Found existing installation: nbformat 4.0.1


    DEPRECATION: Uninstalling a distutils installed project (nbformat) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.


    Uninstalling nbformat-4.0.1:
      Successfully uninstalled nbformat-4.0.1


You are using pip version 8.0.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


0

### Import Libraries

In [98]:
import os # Operation System
import re # Regular Expression
import pandas as pd # Pandas
import numpy as np
import plotly

### Gloabal Variable

In [2]:
url = "C:/Users/rinlin/Desktop/LaborBureauData"
pattern = re.compile("([0-9]+)")

### User Defined Function

In [68]:
 def national_by_year(data,subgroup,numeric="tot_emp"):
    data = data[data['group'] == subgroup]
    data = data[['st','a_mean','tot_emp','occ_title','year']]
    return data.groupby(['year'])[numeric].sum()

### Read In Raw Data

In [3]:
file_collections = []
for root, dirs,files in os.walk(url):
    if len(dirs) == 0:
        data = pd.read_excel(root+"/"+files[1])
        data.columns = [col.lower() for col in data]
        data.rename(columns={"occ_titl":"occ_title"},inplace=True)
        try:
            data.rename(columns={"occ_group":"group"},inplace=True)
        except:
            pass
        data = data[['st','state','a_mean','tot_emp','a_median','occ_title','group']]
        data['year'] = pattern.findall(files[1])[0]
        file_collections.append(data)

In [43]:
all_data = pd.concat(file_collections,ignore_index=False)

### Account For NA

In [44]:
all_data[all_data['tot_emp'] == "**"] = np.nan
all_data[all_data['a_mean'] == "*"] = np.nan
all_data['a_mean'] = all_data['tot_emp'].apply(float)
all_data['tot_emp'] = all_data['tot_emp'].apply(float)

In [47]:
all_data.ix[all_data['group']=="maj",'group'] = "major"

In [48]:
print(all_data.pivot_table(index="year",columns="group",values="tot_emp",aggfunc=[len]))

           len                     
group detailed    div   major total
year                               
1997       NaN  378.0     NaN   NaN
1998       NaN  378.0   701.0   NaN
1999       NaN    NaN  1176.0   NaN
2000       NaN    NaN  1186.0   NaN
2001       NaN    NaN  1186.0   NaN
2002       NaN    NaN  1236.0   NaN
2003       NaN    NaN  1176.0   NaN
2004       NaN    NaN  1181.0   NaN
2005       NaN    NaN  1156.0   NaN
2006       NaN    NaN  1179.0   NaN
2007       NaN    NaN  1181.0   NaN
2008       NaN    NaN  1169.0   NaN
2009       NaN    NaN  1182.0  54.0
2010       NaN    NaN  1180.0  54.0
2011       NaN    NaN  1184.0  54.0
2012   34796.0    NaN  1177.0  54.0
2013   34747.0    NaN  1185.0  54.0
2014   34656.0    NaN  1187.0  54.0
2015   34583.0    NaN  1187.0  54.0
2016   34413.0    NaN  1188.0  54.0


In [58]:
## Total Data
national_by_year(all_data,subgroup="total")

year
2009    131713800.0
2010    128150010.0
2011    129312400.0
2012    131331400.0
2013    133614660.0
2014    136129200.0
2015    138885360.0
2016    141382280.0
Name: tot_emp, dtype: float64


In [59]:
## Total Data
national_by_year(all_data,subgroup="major")

year
1998     86043030.0
1999    128234630.0
2000    130849730.0
2001    129059050.0
2002    257152540.0
2003    128498410.0
2004    129170580.0
2005    126764390.0
2006    133687900.0
2007    135406330.0
2008    135718090.0
2009    131684980.0
2010    127893340.0
2011    129301640.0
2012    131215960.0
2013    133613840.0
2014    136128990.0
2015    138885020.0
2016    141382120.0
Name: tot_emp, dtype: float64


In [60]:
## div Data
national_by_year(all_data,subgroup="div")

year
1997   NaN
1998   NaN
Name: tot_emp, dtype: float64


In [61]:
## detailed Data
national_by_year(all_data,subgroup="detailed")

year
2012    130263850.0
2013    132345780.0
2014    134336710.0
2015    137647860.0
2016    139944710.0
Name: tot_emp, dtype: float64


In [114]:
# it looks like major_data incorporate most of the data with relative accuracy with the exception for year 2002. Therefore we 
# decide to use major_data as our data input, but leaving out year of 2002.
data_valid = all_data[(all_data['group']=="major")&(all_data['year'] !="1998")&(all_data['year'] !="2002")]

In [93]:
sum(np.isnan(data_valid['tot_emp']))

0

In [94]:
sum(np.isnan(data_valid['a_mean']))

0

No NA Value is detected.

### Simple Data Visualization: Year Over Year Trend

In [115]:
data_for_visual=data_valid.groupby(['year'])['tot_emp'].sum().reset_index()

In [116]:
from plotly.offline import *
import plotly.plotly as py
import plotly.graph_objs as go
#from plotly.graph_objs import Scatter, Figure, Layout

plotly.tools.set_credentials_file(username='splashirina', api_key='ErzGtB6zqhc7e8Di37kB')

# Create a trace
trace = go.Scatter(
    x = data_for_visual['year'],
    y = data_for_visual['tot_emp']
)

data = [trace]

py.iplot(data, filename='basic-line')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~splashirina/0 or inside your plot.ly account where it is named 'basic-line'


In [117]:
data_for_visual.to_csv("OES1999_2016.csv",index=False)